In [1]:
# Include necessary imports
import os
import torch 
import pandas as pd
from torch.utils.data import DataLoader
from music21 import *
import numpy as np

In [2]:
# Preprocess the data

folder_path = 'Data/'
test = []
train = []
validation = []
for dirname in os.listdir(folder_path):
    if dirname != '.DS_Store':
        for filename in os.listdir(folder_path + dirname):
            df = pd.read_csv(folder_path + dirname + '/' + filename)
            transposed_df = df.transpose()
            if dirname == 'test':
                test.append(transposed_df)
            if dirname == 'train':
                train.append(transposed_df)
            if dirname == 'valid':
                validation.append(transposed_df)

# Model

In [3]:
class Model(torch.nn.Module):
    def __init__(self, input_size, output_size, hidden_dim=50, n_layers=1):
        super(Model, self).__init__()
        self.hidden_dim = hidden_dim
        self.n_layers = n_layers
        self.lstm = torch.nn.LSTM(input_size, hidden_dim, n_layers, batch_first=True)
        self.fc = torch.nn.Linear(hidden_dim, output_size) # fully connected layer
        
    def forward(self, x, hidden=None):
        lstm_output, (h,c) = self.lstm(x, hidden)
        model_output = self.fc(lstm_output)
        return model_output

# Train

In [4]:
def train_model(model, melody, harmonies, optimizer, criterion, num_epochs):
    for epoch in range(num_epochs):
        optimizer.zero_grad()
        output = model(melody)
        loss = criterion(output, harmonies)
        loss.backward()
        optimizer.step()
        if (epoch + 1) % 100 == 0:
            print("Epoch: ", epoch, "Loss: ", loss)

In [6]:
criterion = torch.nn.MSELoss()
# for song in train:
melody = torch.tensor(train[0].iloc[0], dtype=torch.float32).unsqueeze(0).reshape(1,train[0].shape[1],1)
harmonies = torch.transpose(torch.tensor(train[0].iloc[1:].values, dtype=torch.float32),0,1).unsqueeze(0)
model = Model(1, harmonies.shape[2])
optimizer = torch.optim.Adam(model.parameters(), lr=0.01)
train_model(model, melody, harmonies, optimizer, criterion, 10000)

Epoch:  99 Loss:  tensor(521.1091, grad_fn=<MseLossBackward0>)
Epoch:  199 Loss:  tensor(30.3138, grad_fn=<MseLossBackward0>)
Epoch:  299 Loss:  tensor(11.4256, grad_fn=<MseLossBackward0>)
Epoch:  399 Loss:  tensor(11.1662, grad_fn=<MseLossBackward0>)
Epoch:  499 Loss:  tensor(11.0592, grad_fn=<MseLossBackward0>)
Epoch:  599 Loss:  tensor(10.9628, grad_fn=<MseLossBackward0>)
Epoch:  699 Loss:  tensor(10.8785, grad_fn=<MseLossBackward0>)
Epoch:  799 Loss:  tensor(10.8182, grad_fn=<MseLossBackward0>)
Epoch:  899 Loss:  tensor(10.7671, grad_fn=<MseLossBackward0>)
Epoch:  999 Loss:  tensor(10.6166, grad_fn=<MseLossBackward0>)
Epoch:  1099 Loss:  tensor(10.0631, grad_fn=<MseLossBackward0>)
Epoch:  1199 Loss:  tensor(10.3206, grad_fn=<MseLossBackward0>)
Epoch:  1299 Loss:  tensor(10.0871, grad_fn=<MseLossBackward0>)
Epoch:  1399 Loss:  tensor(10.0125, grad_fn=<MseLossBackward0>)
Epoch:  1499 Loss:  tensor(10.4362, grad_fn=<MseLossBackward0>)
Epoch:  1599 Loss:  tensor(9.1791, grad_fn=<MseLos

In [ ]:
# tiny = pd.DataFrame([[67,62,59,43], [68,62,59,43]]).transpose()
# melody = torch.tensor(tiny.iloc[0], dtype=torch.float32).unsqueeze(0).reshape(1,2,1)
# harmonies = torch.transpose(torch.tensor(tiny.iloc[1:].values, dtype=torch.float32),0,1).unsqueeze(0)
# model = Model(1, harmonies.shape[2])
# optimizer = torch.optim.Adam(model.parameters(), lr=0.01)
# criterion = torch.nn.MSELoss()
# train_model(model, melody, harmonies, optimizer, criterion, 10000)

In [8]:
result = model(torch.tensor(train[0].iloc[0], dtype=torch.float32).unsqueeze(0).reshape(1,train[0].shape[1],1))
print(result)
full_song = converter.parse(result)
full_song.show()
full_song.show('midi')

tensor([[[61.0092, 57.0034, 54.1902],
         [61.4625, 57.8580, 54.5095],
         [62.2583, 58.6631, 55.2584],
         [63.2468, 59.4902, 55.6580],
         [64.2538, 60.3273, 56.0988],
         [64.7233, 60.7189, 56.3480],
         [64.8633, 60.8212, 56.3758],
         [64.9187, 60.8359, 56.2816],
         [64.9722, 60.8040, 56.0001],
         [65.1293, 60.7128, 55.1786],
         [65.5509, 60.4491, 52.8960],
         [66.0622, 60.7578, 52.6673],
         [64.4261, 59.2386, 50.5122],
         [62.8721, 57.8144, 48.5669],
         [62.5976, 57.5629, 48.2244],
         [62.5836, 57.5500, 48.2068],
         [62.5830, 57.5495, 48.2060],
         [62.5830, 57.5495, 48.2060],
         [62.8066, 57.7539, 48.4823],
         [62.8807, 57.8217, 48.5743],
         [63.4153, 58.3105, 49.2373],
         [63.9250, 58.7767, 49.8698],
         [64.4970, 59.2998, 50.5797],
         [65.1108, 59.8611, 51.3413],
         [64.8789, 59.6491, 51.0538],
         [64.6248, 59.4167, 50.7384],
         [64

IndexError: list index out of range

In [ ]:
# Finetune (hyperparameters, move around test data (refer to notes), etc)

In [ ]:
# Test with new data + evaluate

In [ ]:
# Make any other changes

In [ ]:
# Sheet music + audio (musicAI)

In [ ]:
# Create new models if time permits (follow steps 3 - 7)

In [ ]:
# Compare models

In [ ]:
# Front end ** if time permits
# - Interactive sheet music
# - musescore front end??